# Harmonize original SSPs to 2023 start date

Assume ssp245 was "correct" history and re-run from there.

In [ ]:
import datetime
import os
import warnings

import aneris.convenience
import numpy as np
import pandas as pd
import scmdata
from dotenv import load_dotenv
from fair import FAIR, __version__
from fair.interface import fill
from fair.io import read_properties
from tqdm.auto import tqdm
import xarray as xr

In [ ]:
df_in = pd.read_csv(
    "../data/emissions/v1.4.0/historical_ssp245.csv"
)

In [ ]:
variables = list(df_in["variable"])
units = list(df_in["unit"])
var_units = {var: unit for var, unit in zip(variables, units)}

times = []
years = range(1750, 2023)
for year in years:
    times.append(datetime.datetime(year, 1, 1))
    # they are really midyears, but we just want this to work

times_future = []
years_future = range(2022, 2500)
for year in years_future:
    times_future.append(datetime.datetime(year, 1, 1))

In [ ]:
history = (
    scmdata.ScmRun(
        "../data/emissions/v1.4.0/historical_ssp245.csv",
        lowercase_cols=True,
    )
    .filter(region="World", variable=variables)
    .interpolate(target_times=times)
    .timeseries(time_axis="year")
)

In [ ]:
history

In [ ]:
future = (
    scmdata.ScmRun(
        "../data/emissions/v1.4.0/future.csv",
        lowercase_cols=True,
    )
    .filter(variable=variables, region="World")
    .interpolate(times_future)
    .timeseries(time_axis="year")
)

In [ ]:
future

In [ ]:
# Harmonization overrides - use same as RCMIP
overrides = pd.DataFrame(
    [
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|C2F6",
        },  # high historical variance (cov=16.2)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|C6F14",
        },  # high historical variance (cov=15.4)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|CF4",
        },  # high historical variance (cov=11.2)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|CO",
        },  # high historical variance (cov=15.4)
        {
            "method": "reduce_ratio_2080",
            "variable": "Emissions|CO2",
        },  # always ratio method by choice
        {
            "method": "reduce_offset_2150_cov",
            "variable": "Emissions|CO2|AFOLU",
        },  # high historical variance, but using offset method to prevent diff from
        # increasing when going negative rapidly (cov=23.2)
        {
            "method": "reduce_ratio_2080",  # always ratio method by choice
            "variable": "Emissions|CO2|Energy and Industrial Processes",
        },
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|CH4'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-125",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-134a",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-143a",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-227ea",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-23",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-32",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-4310mee",
        },  # minor f-gas with low model reporting confidence
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|N2O'},
        # depending on the decision tree in aneris/method.py
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|NH3'},
        # depending on the decision tree in aneris/method.py
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|NOx'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|OC",
        },  # high historical variance (cov=18.5)
        {
            "method": "constant_ratio",
            "variable": "Emissions|SF6",
        },  # minor f-gas with low model reporting confidence
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|Sulfur'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|VOC",
        },  # high historical variance (cov=12.0)
    ]
)

harmonisation_year = 2022

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    scenarios_harmonised = [
        aneris.convenience.harmonise_all(
            msdf,
            history=history,
            harmonisation_year=harmonisation_year,
            overrides=overrides,
        ).reset_index(level=(5, 6, 7, 8, 9), drop=True)
        for _, msdf in tqdm(future.groupby(["model", "scenario"]))
    ]

In [ ]:
scenarios_harmonised

In [ ]:
scenarios_harmonised = pd.concat(scenarios_harmonised).reset_index()

os.makedirs(
    "../output/emissions/v1.4.0",
    exist_ok=True,
)

scenarios_harmonised.to_csv(
    "../output/emissions/v1.4.0/ssps_harmonized_2022-2499.csv",
    index=False,
)

In [ ]:
scenarios = [
    "ssp119",
    "ssp126",
    "ssp245",
    "ssp370",
    "ssp434",
    "ssp460",
    "ssp534-over",
    "ssp585",
]

In [ ]:
fair_map = {var: var.split("|")[-1] for var in variables}
fair_map["Emissions|CO2|Energy and Industrial Processes"] = "CO2 FFI"
fair_map["Emissions|CO2|AFOLU"] = "CO2 AFOLU"

In [ ]:
ssp245_in = pd.read_csv("../data/emissions/v1.4.0/ssp245.csv")

In [ ]:
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")

f = FAIR(ch4_method="thornhill2021")
f.define_time(1750, 2500, 1)
f.define_configs(["unspecified"])
f.define_scenarios(scenarios)
f.define_species(species, properties)

f.allocate()
f.fill_from_rcmip()

In [ ]:
# fill emissions
for scenario in scenarios:
    for specie in fair_map:
        data_his = history.xs(specie, level="variable").loc[:, 1750:2021].values.squeeze()
        if scenario != "ssp245":
            data_fut = scenarios_harmonised.loc[
                (
                    (scenarios_harmonised["scenario"]==scenario) &
                    (scenarios_harmonised["variable"]==specie)
                ), 2022:2499].values.squeeze()
        else:
            data_fut = ssp245_in.loc[ssp245_in["variable"] == specie, "2022":"2499"].values.squeeze()
        data = np.concatenate((data_his, data_fut))
        fill(
            f.emissions,
            data,
            config="unspecified",
            scenario=scenario,
            #        timepoints=np.arange(1750.5, 2021),
            specie=fair_map[specie],
        )

In [ ]:
f.emissions

In [ ]:
f.emissions.to_netcdf(
    "../output/emissions/v1.4.0/ssps_2022_harmonization_1750-2499.nc"
)